In [ ]:
from models.UNet import UNet
from models.RUNet import RUNet

import torch
from PIL import Image
from torchvision.transforms import Grayscale,ToTensor,Resize,GaussianBlur,ToPILImage
from torchvision.utils import save_image
from utils import lr_transform_valid, psnr,ssim
import numpy as np

model = torch.jit.load("test_scripted.pt")

In [ ]:
model = torch.load("test_model.pt")
model.eval()

In [ ]:
model = RUNet().to("cpu")
model.load_state_dict(torch.load("./savedModels/RUNET_PLS_PERCEPTUAL.pth",map_location=torch.device('cpu')))

In [ ]:
model.eval()
IMAGE = "./datasets/tests/comics.jpg"

in_image = Image.open(IMAGE)
in_image = Resize(in_image.size[0] // 2)(Grayscale()(in_image))

interpolated =  lr_transform((in_image.size[1],in_image.size[0]),2)(ToTensor()(in_image))

in_image = ToTensor()(in_image)
# in_image += (0.01**0.5)*torch.rand_like(in_image)
save_image(in_image,"./outputs/one_image_input.jpg")
save_image(interpolated,"./outputs/one_image_interpolation.jpg")
out = model(interpolated.unsqueeze(0))
save_image(out,"./outputs/one_image_out.jpg")

interpolated = interpolated.unsqueeze(0)
in_image = in_image.unsqueeze(0)

print("SSIM")
print("interpolation: ",ssim(interpolated,in_image))
print("net: ",ssim(out,in_image))

print("PSNR")
print("interpolation: ",psnr(interpolated,in_image))
print("net: ",psnr(out,in_image))
